<a href="https://colab.research.google.com/github/Poojashrees3/Statistics-and-machine-learning/blob/Mybranch/Exercise_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import shap


data = pd.read_csv("epi_r.csv" )
data.head()


,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**handling missing values**

In [3]:
# Checking for missing values
missing_data = data.isnull().sum()
print(missing_data[missing_data > 0])
missing_data_columns= missing_data[missing_data > 0].index

calories    4117
protein     4162
fat         4183
sodium      4119
dtype: int64


**Cleaning data**

In [4]:
# data cleaning done by imputation method
numeric_data = data[missing_data_columns].select_dtypes(include=np.number)
data[numeric_data.columns] = numeric_data.fillna(numeric_data.mean(), inplace=False)
print(data[numeric_data.columns])

data = data.dropna(subset=['rating'])

data.head()

          calories     protein         fat       sodium
0       426.000000   30.000000    7.000000   559.000000
1       403.000000   18.000000   23.000000  1439.000000
2       165.000000    6.000000    7.000000   165.000000
3      6322.958017  100.160793  346.877497  6225.974895
4       547.000000   20.000000   32.000000   452.000000
...            ...         ...         ...          ...
20047    28.000000    2.000000    2.000000    64.000000
20048   671.000000   22.000000   28.000000   583.000000
20049   563.000000   31.000000   38.000000   652.000000
20050   631.000000   45.000000   24.000000   517.000000
20051   560.000000   73.000000   10.000000  3698.000000

[20052 rows x 4 columns]


,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.000000,30.000000,7.000000,559.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.000000,18.000000,23.000000,1439.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.000000,6.000000,7.000000,165.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,6322.958017,100.160793,346.877497,6225.974895,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.000000,20.000000,32.000000,452.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**1.Tree Models**

**1.1 Decision Tree Classifier with Evaluation metrics**

In [5]:
X = data.drop('rating', axis=1)
y = data['rating']

#categorising data
y = pd.cut(y, bins=[0, 3, 4, 5], labels=['low', 'medium', 'high'])

# Converting non-numeric columns to numerical using one-hot encoding
X = pd.get_dummies(X)


# The y values might have NaNs, leading to the error
# Drop rows with NaNs in 'y' before splitting
X_train, X_test, y_train, y_test = train_test_split(X[~y.isna()], y[~y.isna()], test_size=0.2, random_state=42)


# Initialize and train a Decision Tree Classifier
tree_classifier = DecisionTreeClassifier(random_state=42) # You can tune hyperparameters here
tree_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = tree_classifier.predict(X_test)

#evaluation metrics

print("Decision Tree Classifier Results")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred,average='weighted'))
print("Recall:", recall_score(y_test, y_pred,average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred,average='weighted'))


Decision Tree Classifier Results
Accuracy: 0.6034577387486278
Precision: 0.5948116489192103
Recall: 0.6034577387486278
F1 Score: 0.5977092353157999


**1.2 Random forest classifier with evaluation metrics**

In [6]:
# Initialize and train a Random forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)  # Initialize
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

#evaluation metrics

print("Random Forest Classifier Results")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred,average='weighted'))
print("Recall:", recall_score(y_test, y_pred,average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred,average='weighted'))

Random Forest Classifier Results
Accuracy: 0.6726125137211855
Precision: 0.6952304581634051
Recall: 0.6726125137211855
F1 Score: 0.6291953593119739


**1.3 Gradient Boosting Classifier with Evaluation metrics**



In [6]:
gb_classifier = GradientBoostingClassifier(n_estimators=100,random_state=42)

# Train the model
gb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = gb_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

#evaluation metrics
print("Gradient Boosting Classifier Results")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred,average='weighted'))
print("Recall:", recall_score(y_test, y_pred,average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred,average='weighted'))

KeyboardInterrupt: 

**2.KNN with different kk-values with plot**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
X = data.drop('rating', axis=1)
y = data['rating']

#categorising data
y = pd.cut(y, bins=[0, 3, 4, 5], labels=['low', 'medium', 'high'])

# Converting non-numeric columns to numerical using one-hot encoding
X = pd.get_dummies(X)


# The y values might have NaNs, leading to the error
# Drop rows with NaNs in 'y' before splitting
X_train, X_test, y_train, y_test = train_test_split(X[~y.isna()], y[~y.isna()], test_size=0.2, random_state=42)

# Find optimal k
k_values = range(1, 10)
accuracy_scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=5,algorithm='kd_tree', leaf_size=30)
    scores = cross_val_score(knn, X_train, y_train, cv=5) # 5-fold cross-validation
    accuracy_scores.append(scores.mean())

# Plot the performance
plt.figure(figsize=(10, 6))
plt.plot(k_values, accuracy_scores)
plt.xlabel("k (Number of Neighbors)")
plt.ylabel("Accuracy")
plt.title("KNN Performance with Different k Values")
plt.grid(True)
plt.show()

# Find the optimal k
optimal_k = k_values[np.argmax(accuracy_scores)]
print(f"Optimal k value: {optimal_k}")

# Train and evaluate the model with the optimal k
knn_optimal = KNeighborsClassifier(n_neighbors=optimal_k)
knn_optimal.fit(X_train, y_train)
y_pred = knn_optimal.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy with optimal k ({optimal_k}): {accuracy}")